# 필요 모듈

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
import zipfile #파일 압축 풀 모듈
from tqdm import tqdm
import unicodedata
import re

import shutil
import cv2
import torch
import os
import numpy as np
from torchvision import transforms

import json # json 파일 읽는 모듈
from collections import Counter # 프레임 내의 리스트중 감정빈도가 제일 높은 경우를 뽑는 모듈
import pickle


from einops import rearrange
import torch.nn as nn

In [ ]:
# 압축 파일 경로
file_path = '//content//drive//MyDrive//sns_multimodal_sentiment_pr'

# 표정데이터셋 다운로드

In [ ]:
#압축파일 다운로드용 poc 이후에 함수로 만들어서 깔끔하게 사용하자

# zip 파일 열기
with zipfile.ZipFile(file_path + '//0~9_감정분류_데이터셋.zip', 'r') as zip_ref:
    # 압축 파일 내의 파일 목록 가져오기
    file_list = zip_ref.namelist()

    # .m2ts 파일만 필터링
    face_files = [file for file in file_list if file.endswith('.m2ts')]
    print(f"추출할 비디오 파일 목록: {face_files}")

    # 비디오 파일을 저장할 폴더 생성
    extract_video_path = file_path + '//face_files'
    os.makedirs(extract_video_path, exist_ok=True)

    for video_file in video_files:
        zip_ref.extract(video_file, extract_video_path)
        print(f"{video_file} 파일을 추출했습니다.")

print("표정비디오 파일 추출이 완료되었습니다.")

# 액션 데이터셋 다운로드

In [ ]:
# zip 파일 열기
with zipfile.ZipFile(file_path + '//0001-0400.zip', 'r') as zip_ref:
    # 압축 파일 내의 파일 목록 가져오기
    file_list = zip_ref.namelist()

    # .m2ts 파일만 필터링
    video_files = [file for file in file_list if file.endswith('.mp4') or file.endswith('.json')]
    print(f"추출할 비디오 파일 목록: {video_files}")

    # 비디오 파일을 저장할 폴더 생성
    extract_video_path = file_path + '//real_action_files'
    os.makedirs(extract_video_path, exist_ok=True)

    for video_file in video_files:
        zip_ref.extract(video_file, extract_video_path)
        print(f"{video_file} 파일을 추출했습니다.")

print("액션비디오 파일과 json 추출이 완료되었습니다.")

# 표정 데이터 전처리

## 파일제목에 감정라벨링

In [ ]:
# 파일 위치 이동으로 사용시 수정이 필요
'''
해야 할 일
비디오 파일 먼저 라벨링부터 매겨두기: 파일이름 맨앞부분을 감정 라벨을 붙여두자

- 감정분류

    1~50 : 행복

    51~100 : 놀람

    101~150 : 중립

    151~200 : 공포

    201~250 : 혐오

    251~300 : 분노

    301~350 : 슬픔

 video file안에

    분노 (Anger)	슬픔 (Sad)	행복 (Joy)	혐오 (Disgust)	공포 (Fear)	놀람 (Surprise)	중립 (Neutral)

'''


happy_cnt = 0  # 총 비디오 데이터들의 표정 데이터를 분석하기 위함 남자 여자별로도 나중에 세보자
surp_cnt = 0
neutral_cnt = 0
fear_cnt = 0
disgust_cnt = 0
anger_cnt = 0
sad_cnt = 0
total_cnt = 0

directory = f'//content//drive//MyDrive//sns_multimodal_sentiment_pr//face_files//all_face_file'  # 파일들이 있는 디렉토리 경로
for filename in os.listdir((directory)):
    filename_normalized = unicodedata.normalize('NFC', filename) # 한글 문자열 인식 위해 변환
    # 001~050 범위는 '행복', 051~100 범위는 '놀람'
    total_cnt += 1 # 전체 비디오 개수 체크
    if '행복' in filename_normalized :
        happy_cnt += 1
    elif '놀람' in filename_normalized :
        surp_cnt += 1
    elif '중립' in filename_normalized :
        neutral_cnt += 1
    elif '공포' in filename_normalized :
        fear_cnt += 1
    elif '혐오' in filename_normalized :
        disgust_cnt += 1
    elif '분노' in filename_normalized :
        anger_cnt += 1
    elif '슬픔' in filename_normalized :
        sad_cnt += 1
    else:
        continue  # 범위 밖의 파일은 변경하지 않음


# 감정 별 비율 개수
print(f'행복:{happy_cnt}\n놀람:{surp_cnt}\n중립:{neutral_cnt}\n공포:{fear_cnt}\n혐오:{disgust_cnt}\n분노:{anger_cnt}\n슬픔:{sad_cnt}\n총 개수{total_cnt}')

행복:136
놀람:149
중립:154
공포:155
혐오:155
분노:156
슬픔:156
총 개수1061


## 표정 데이터 프레임 분할 및 annotaion



In [ ]:
# 감정 라벨을 추출하는 함수 (파일명에서)
def extract_emotion_from_filename(filename):
    # 예: "009-혐오68.m2ts"에서 "혐오"를 추출
    pattern = re.compile(r"-(\D+)\d*")  # 숫자 이외의 문자를 추출하는 정규 표현식
    match = pattern.search(filename)
    if match:
        return match.group(1)  # 감정 라벨 반환
    else:
        return "unknown"  # 감정 라벨이 없을 경우 처리

# 표정 비디오 파일에서 프레임을 저장하고 감정 라벨을 피클 파일로 저장하는 함수
def process_expression_video(video_path, output_frames_dir, pickle_save_path, target_size=(224, 224)):
    # 비디오 파일 제목에서 감정 라벨 추출
    video_filename = os.path.basename(video_path)  # 비디오 파일명만 추출
    emotion_label = extract_emotion_from_filename(video_filename)  # 파일명에서 감정 라벨 추출
    print(f"Processing video {video_path}: Emotion Label - {emotion_label}")

    # 비디오 파일 열기
    video = cv2.VideoCapture(video_path)

    # 비디오의 총 프레임 수 확인
    total_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = video.get(cv2.CAP_PROP_FPS)
    duration = total_frames / fps
    print(f"Total frames: {total_frames}, FPS: {fps}, Duration: {duration} seconds")

    # 각 프레임에 대한 감정 라벨을 저장할 딕셔너리
    frame_idx = 0
    emotion_dict = {}

    # 프레임을 처리하며 저장
    while video.isOpened():
        ret, frame = video.read()  # 프레임 읽기
        if not ret:
            break  # 더 이상 프레임이 없으면 종료

        # 프레임의 크기를 224x224로 조정
        resized_frame = cv2.resize(frame, target_size)

        # 프레임 저장 경로 설정
        frame_filename = f"frame_{frame_idx}.jpg"
        frame_output_path = os.path.join(output_frames_dir, frame_filename)

        # 디렉토리가 없으면 생성
        os.makedirs(output_frames_dir, exist_ok=True)

        # 프레임을 이미지 파일로 저장
        cv2.imwrite(frame_output_path, resized_frame)

        # 프레임에 해당하는 감정 라벨 저장
        emotion_dict[frame_idx] = emotion_label

        frame_idx += 1  # 다음 프레임으로

    video.release()  # 비디오 파일 닫기

    # 감정 라벨을 피클 파일로 저장
    os.makedirs(os.path.dirname(pickle_save_path), exist_ok=True)  # 피클 저장 경로가 없으면 생성
    with open(pickle_save_path, 'wb') as f:
        pickle.dump(emotion_dict, f)

    print(f"Emotion labels saved to {pickle_save_path} for video {video_path}.")

# 사용 예시 액션 비디오 클립 개수 보고 표정 클립 개수도 체크하기

directory = "//content//drive//MyDrive//sns_multimodal_sentiment_pr//face_files//all_face_file"  # 비디오 파일이 있는 디렉토리 경로

for index, filename in tqdm(enumerate(os.listdir(directory))):

    output_dir_2 = f'//content//drive//MyDrive//sns_multimodal_sentiment_pr//total_video_output_frame_files//clip{399 + index}_frame'  # 프레임 저장 경로
    pickle_save_path_2 = f'//content//drive//MyDrive//sns_multimodal_sentiment_pr//total_emotion_label_pkl//emotion_labels_clip{399 + index}.pkl'  # 피클 저장 경로

    # 비디오 처리 함수 호출
    video_path = os.path.join(directory, filename)
    process_expression_video(video_path, output_dir_2, pickle_save_path_2)

0it [00:00, ?it/s]

Processing video //content//drive//MyDrive//sns_multimodal_sentiment_pr//face_files//all_face_file/000-혐오61.m2ts: Emotion Label - 혐오
Total frames: 151, FPS: 29.97002997002997, Duration: 5.038366666666667 seconds


1it [00:02,  2.64s/it]

Emotion labels saved to //content//drive//MyDrive//sns_multimodal_sentiment_pr//total_emotion_label_pkl//emotion_labels_clip399.pkl for video //content//drive//MyDrive//sns_multimodal_sentiment_pr//face_files//all_face_file/000-혐오61.m2ts.
Processing video //content//drive//MyDrive//sns_multimodal_sentiment_pr//face_files//all_face_file/000-혐오62.m2ts: Emotion Label - 혐오
Total frames: 151, FPS: 29.97002997002997, Duration: 5.038366666666667 seconds


2it [00:05,  3.02s/it]

...

Emotion labels saved to //content//drive//MyDrive//sns_multimodal_sentiment_pr//total_emotion_label_pkl//emotion_labels_clip1457.pkl for video //content//drive//MyDrive//sns_multimodal_sentiment_pr//face_files//all_face_file/000-혐오58.m2ts.
Processing video //content//drive//MyDrive//sns_multimodal_sentiment_pr//face_files//all_face_file/000-혐오59.m2ts: Emotion Label - 혐오
Total frames: 151, FPS: 29.97002997002997, Duration: 5.038366666666667 seconds


1060it [56:07,  3.52s/it]

Emotion labels saved to //content//drive//MyDrive//sns_multimodal_sentiment_pr//total_emotion_label_pkl//emotion_labels_clip1458.pkl for video //content//drive//MyDrive//sns_multimodal_sentiment_pr//face_files//all_face_file/000-혐오59.m2ts.
Processing video //content//drive//MyDrive//sns_multimodal_sentiment_pr//face_files//all_face_file/000-혐오60.m2ts: Emotion Label - 혐오
Total frames: 151, FPS: 29.97002997002997, Duration: 5.038366666666667 seconds


1061it [56:11,  3.18s/it]

Emotion labels saved to //content//drive//MyDrive//sns_multimodal_sentiment_pr//total_emotion_label_pkl//emotion_labels_clip1459.pkl for video //content//drive//MyDrive//sns_multimodal_sentiment_pr//face_files//all_face_file/000-혐오60.m2ts.


# 액션 데이터 전처리
## 액션 데이터 프레임 분할 및 annotation
### 감정 라벨이 없는 프레임은 이전 프레임의 감정 라벨을 따라 보간 처리

In [ ]:
# 감정 라벨 보간 함수
def interpolate_emotions(emotion_dict, total_frames):
    # 보간할 라벨 딕셔너리
    interpolated_emotion_dict = emotion_dict.copy()

    # 첫 번째부터 연속된 no_emotion 프레임 처리: 다음 감정이 있는 프레임을 찾아서 그 감정으로 보간
    if interpolated_emotion_dict.get(0) == 'no_emotion':
        first_non_emotion_idx = None
        for idx in range(1, total_frames):
            if interpolated_emotion_dict.get(idx) != 'no_emotion':
                first_non_emotion_idx = idx
                break
        if first_non_emotion_idx is not None:
            # 첫 번째 감정이 있는 프레임을 찾은 경우 그 감정으로 첫 번째부터 해당 구간까지 보간
            for i in range(first_non_emotion_idx):
                interpolated_emotion_dict[i] = interpolated_emotion_dict[first_non_emotion_idx]

    # 감정 라벨이 없는 'no_emotion' 구간을 보간
    start_idx = None
    for idx in range(total_frames):
        if interpolated_emotion_dict.get(idx) == 'no_emotion':
            if start_idx is None:
                start_idx = idx
        else:
            if start_idx is not None:
                end_idx = idx
                # 앞뒤 감정을 기반으로 보간할 감정을 결정
                prev_emotion = interpolated_emotion_dict.get(start_idx - 1, 'no_emotion')
                next_emotion = interpolated_emotion_dict.get(end_idx, 'no_emotion')

                if prev_emotion == next_emotion:
                    # 앞뒤 감정이 같으면 그 감정을 중간 프레임에 적용
                    fill_emotion = prev_emotion
                else:
                    # 앞뒤 감정이 다르면, 기본적으로 앞의 감정을 중간 프레임에 적용
                    fill_emotion = prev_emotion

                # no_emotion 구간을 선택된 감정으로 채움
                for i in range(start_idx, end_idx):
                    interpolated_emotion_dict[i] = fill_emotion

                start_idx = None

    # 마지막 프레임이 no_emotion일 경우 처리
    if start_idx is not None:
        # 이전 프레임의 감정을 마지막 no_emotion 구간에 적용
        prev_emotion = interpolated_emotion_dict.get(start_idx - 1, 'no_emotion')
        for i in range(start_idx, total_frames):
            interpolated_emotion_dict[i] = prev_emotion

    return interpolated_emotion_dict

# 감정 라벨 추출 및 프레임 저장 함수
def extract_emotion_and_save_frames(video_path, json_path, output_dir, pickle_save_path, target_size=(224, 224)):
    # JSON 파일 로드 (비디오와 매칭되는 감정 데이터를 포함한 json 파일)
    with open(json_path, 'r') as f:
        data = json.load(f)

    # 비디오 파일 로드 (OpenCV를 통해 비디오 파일을 열고 프레임 단위로 접근할 수 있음)
    video = cv2.VideoCapture(video_path)

    # 비디오의 총 프레임 수와 fps 계산
    total_video_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = video.get(cv2.CAP_PROP_FPS)
    duration = total_video_frames / fps  # 비디오 길이 (초)
    print(f"Total frames: {total_video_frames}, FPS: {fps}, Duration: {duration}")

    frame_idx = 0  # 처리할 프레임 인덱스 초기화
    emotion_dict = {}  # 프레임 번호와 감정을 저장할 딕셔너리
    all_no_emotion = True  # 감정이 모두 no_emotion인지 확인하는 플래그

    # 비디오를 프레임 단위로 처리
    while video.isOpened():
        ret, frame = video.read()  # 프레임을 읽음
        if not ret:  # 프레임을 더 이상 읽을 수 없다면 루프 종료
            break

        # JSON에서 현재 json_frame_idx에 해당하는 데이터가 있으면 처리
        if str(frame_idx) in data['data']:
            frame_data = data['data'][str(frame_idx)]  # 현재 프레임 번호에 해당하는 데이터
            emotion_list = []  # 프레임별 사람 객체의 감정 리스트

            # 프레임 안의 각 객체에 대해 처리
            for key in frame_data.keys():
                if 'emotion' in frame_data[key] and 'image' in frame_data[key]['emotion'] and 'emotion' in frame_data[key]['emotion']['image']:
                    emotion = frame_data[key]['emotion']['image']['emotion']  # 감정 추출
                else:
                    emotion = "no_emotion"  # 감정이 없으면 no_emotion으로 처리
                emotion_list.append(emotion)  # 감정 리스트에 추가

            # 해당 프레임의 감정들 중 빈도가 가장 높은 감정을 선택
            counter = Counter(emotion_list)

            if len(counter) == 1 and 'no_emotion' in counter:  # 감정이 모두 'no_emotion'인 경우
                most_common_emotion = 'no_emotion'
            else:
                most_common_emotion = counter.most_common(1)[0][0]  # 가장 많이 등장한 감정을 선택
                all_no_emotion = False  # 감정이 하나라도 no_emotion이 아닐 경우 플래그 False로 변경

            # 해당 프레임의 감정 저장
            emotion_dict[frame_idx] = most_common_emotion

            # 프레임 해상도를 224x224로 축소
            resized_frame = cv2.resize(frame, target_size)

            # 프레임 파일 저장 경로 설정 (output 폴더에 프레임 저장)
            frame_file = os.path.join(output_dir, f"frame_{frame_idx}.jpg")
            os.makedirs(os.path.dirname(frame_file), exist_ok=True)  # 디렉토리 생성 (경로가 없으면 생성)
            cv2.imwrite(frame_file, resized_frame)  # 프레임 이미지를 해당 경로에 저장

        frame_idx += 1  # 다음 프레임으로 넘어감

    video.release()  # 비디오 파일 닫기

    # 모든 프레임의 감정이 no_emotion이면 해당 비디오 저장하지 않음
    if all_no_emotion:
        print(f"Video {video_path} skipped: all frames are 'no_emotion'")
        return False  # 이 비디오 클립은 저장하지 않음

    # no_emotion 구간 보간 처리
    emotion_dict = interpolate_emotions(emotion_dict, total_video_frames)

    # 감정 라벨을 피클 파일로 저장
    os.makedirs(os.path.dirname(pickle_save_path), exist_ok=True)  # 디렉토리가 없으면 생성
    with open(pickle_save_path, 'wb') as f:  # 피클 파일로 감정 딕셔너리 저장
        pickle.dump(emotion_dict, f)

    print(f"Emotion labels saved to {pickle_save_path} for video {video_path}.")
    return True  # 비디오 클립 저장됨

# 클립을 처리하는 함수 후에 end_clip 400까지 늘리기
def process_video_clips(start_clip=1, end_clip=400, base_dir="//content//drive//MyDrive//sns_multimodal_sentiment_pr//action_files//0001-0400"):
    saved_clip_idx = 1  # 삭제된 클립을 제외하고 저장할 클립 번호
    for i in tqdm(range(start_clip, end_clip + 1)):
        video_path = f'{base_dir}//clip_{i}//clip_{i}.mp4'
        json_path = f'{base_dir}//clip_{i}//clip_{i}.json'
        output_dir = f'/content/drive/MyDrive/sns_multimodal_sentiment_pr//total_video_output_frame_files//clip{saved_clip_idx}_frame'  # 프레임 저장 경로
        pickle_save_path = f'//content//drive//MyDrive//sns_multimodal_sentiment_pr//total_emotion_label_pkl//emotion_labels_clip{saved_clip_idx}.pkl'  # 피클 저장 경로

        # 비디오 클립과 감정 데이터를 처리하고 저장 여부 확인
        if extract_emotion_and_save_frames(video_path, json_path, output_dir, pickle_save_path):
            saved_clip_idx += 1  # 저장된 클립만 번호 증가

# 사용 예시: 1번부터 50번 클립까지 처리
process_video_clips(start_clip= 1, end_clip=400)











  0%|          | 0/1 [00:00<?, ?it/s]

Total frames: 1740, FPS: 29.97, Duration: 58.05805805805806


100%|██████████| 1/1 [01:07<00:00, 67.34s/it]

Emotion labels saved to //content//drive//MyDrive//sns_multimodal_sentiment_pr//total_emotion_label_pkl//emotion_labels_clip10.pkl for video //content//drive//MyDrive//sns_multimodal_sentiment_pr//action_files//0001-0400//clip_10//clip_10.mp4.


In [ ]:
# 피클 파일 내의 'neutral' 라벨이 포함되지 않은 피클 파일의 개수를 출력하는 함수
def count_non_neutral_pickle_files(pickle_directory_path):
    non_neutral_count = 0
    all_pickle_files = sorted(os.listdir(pickle_directory_path))

    for filename in all_pickle_files:
        if filename.endswith('.pkl'):
            pickle_file_path = os.path.join(pickle_directory_path, filename)

            try:
                with open(pickle_file_path, 'rb') as f:
                    data = pickle.load(f)

                # 피클 파일에서 'neutral' 라벨이 포함되지 않은 경우 카운트
                if 'neutral' not in data.values():
                    non_neutral_count += 1

            except (pickle.UnpicklingError, EOFError, AttributeError, ValueError) as e:
                print(f"Error reading pickle file {pickle_file_path}: {e}")
                continue

    return non_neutral_count

# 사용 예시
pickle_directory_path = '//content//drive//MyDrive//sns_multimodal_sentiment_pr//total_emotion_label_pkl'

# 'neutral' 라벨이 포함되지 않은 피클 파일의 개수 출력
non_neutral_count = count_non_neutral_pickle_files(pickle_directory_path)
print(f"'neutral' 라벨이 포함되지 않은 피클 파일의 개수: {non_neutral_count}")

'neutral' 라벨이 포함되지 않은 피클 파일의 개수: 144


In [ ]:
# 피클 파일 내에서 'neutral'과 'no_emotion' 라벨이 포함되지 않은 피클 파일의 개수를 출력하는 함수
def count_non_neutral_no_emotion_pickle_files(pickle_directory_path):
    non_neutral_no_emotion_count = 0
    all_pickle_files = sorted(os.listdir(pickle_directory_path))

    for filename in all_pickle_files:
        if filename.endswith('.pkl'):
            pickle_file_path = os.path.join(pickle_directory_path, filename)

            try:
                with open(pickle_file_path, 'rb') as f:
                    data = pickle.load(f)

                # 피클 파일에서 'neutral' 또는 'no_emotion' 라벨이 포함되지 않은 경우 카운트
                if 'neutral' not in data.values() and 'no_emotion' not in data.values():
                    non_neutral_no_emotion_count += 1

            except (pickle.UnpicklingError, EOFError, AttributeError, ValueError) as e:
                print(f"Error reading pickle file {pickle_file_path}: {e}")
                continue

    return non_neutral_no_emotion_count

# 사용 예시
pickle_directory_path = '//content//drive//MyDrive//sns_multimodal_sentiment_pr//total_emotion_label_pkl'

# 'neutral'과 'no_emotion' 라벨이 포함되지 않은 피클 파일의 개수 출력
non_neutral_no_emotion_count = count_non_neutral_no_emotion_pickle_files(pickle_directory_path)
print(f"'neutral'과 'no_emotion' 라벨이 포함되지 않은 피클 파일의 개수: {non_neutral_no_emotion_count}")

In [ ]:
0# no_emotion이 포함된 피클 파일의 개수를 세는 함수
def count_no_emotion_pickle_files(pickle_directory):
    no_emotion_count = 0  # no_emotion이 포함된 파일 개수 카운트

    # 피클 파일이 있는 디렉토리의 모든 파일에 대해 처리
    for filename in os.listdir(pickle_directory):
        if filename.endswith('.pkl'):  # 피클 파일만 처리
            pickle_file_path = os.path.join(pickle_directory, filename)

            try:
                # 피클 파일 읽기
                with open(pickle_file_path, 'rb') as f:
                    emotion_data = pickle.load(f)

                # no_emotion이 포함된 경우 카운트 증가
                if 'no_emotion' in emotion_data.values():
                    no_emotion_count += 1
                    print(f"File {filename} contains 'no_emotion'.")

            except Exception as e:
                print(f"Error processing {pickle_file_path}: {e}")

    print(f"\nTotal number of files containing 'no_emotion': {no_emotion_count}")
    return no_emotion_count

# 사용 예시
pickle_directory = '/content/drive/MyDrive/sns_multimodal_sentiment_pr/total_emotion_label_pkl'

# no_emotion이 포함된 피클 파일 개수 세기
count_no_emotion_pickle_files(pickle_directory)

In [ ]:
# 감정 라벨 보간 함수 (앞부분이 no_emotion인 경우 이후 감정으로 보간)
def interpolate_no_emotion_at_start(emotion_dict, total_frames):
    # 보간할 라벨 딕셔너리 (원본 딕셔너리 복사)
    interpolated_emotion_dict = emotion_dict.copy()

    start_idx = None
    first_non_emotion_idx = None

    # no_emotion 구간을 찾음
    for idx in range(1, total_frames + 1):  # 프레임 번호는 1부터 시작
        if interpolated_emotion_dict.get(idx) == 'no_emotion':
            if start_idx is None:
                start_idx = idx  # no_emotion 구간 시작
        else:
            if start_idx is not None:
                first_non_emotion_idx = idx  # 첫 번째로 감정이 있는 프레임을 찾음
                break

    # no_emotion 구간을 보간 처리: 이후 감정으로 채움
    if first_non_emotion_idx is not None and start_idx is not None:
        for i in range(start_idx, first_non_emotion_idx):  # 시작점부터 첫 감정까지 no_emotion을 감정으로 채움
            interpolated_emotion_dict[i] = interpolated_emotion_dict[first_non_emotion_idx]
        print(f"Interpolated no_emotion from frame {start_idx} to {first_non_emotion_idx - 1} using frame {first_non_emotion_idx} (emotion: {interpolated_emotion_dict[first_non_emotion_idx]})")
    return interpolated_emotion_dict

# 전부 no_emotion인 피클 파일을 삭제하고, 매칭되는 비디오 클립도 삭제하는 함수
def delete_no_emotion_only_files(pickle_file_path, frame_directory_path):
    # 매칭되는 비디오 클립 디렉토리 경로 추정 (클립 번호를 추출해서 매칭)
    clip_number = ''.join(filter(str.isdigit, os.path.basename(pickle_file_path)))
    frame_dir = os.path.join(frame_directory_path, f"clip{clip_number}_frame")

    # 피클 파일 삭제
    print(f"Deleting pickle file: {pickle_file_path}")
    os.remove(pickle_file_path)

    # 매칭되는 비디오 클립 디렉토리 삭제
    if os.path.exists(frame_dir):
        print(f"Deleting video frame directory: {frame_dir}")
        shutil.rmtree(frame_dir)

# 디렉토리 내 모든 피클 파일을 처리하는 함수
def process_pickle_files_in_directory(pickle_directory, frame_directory):
    for filename in os.listdir(pickle_directory):
        if filename.endswith('.pkl'):  # 피클 파일만 처리
            pickle_file_path = os.path.join(pickle_directory, filename)

            try:
                # 피클 파일 읽기
                with open(pickle_file_path, 'rb') as f:
                    emotion_data = pickle.load(f)

                total_frames = max(emotion_data.keys())  # 총 프레임 수 추정

                # 모든 감정이 no_emotion인 경우 파일과 매칭되는 비디오 클립을 삭제
                if set(emotion_data.values()) == {'no_emotion'}:
                    delete_no_emotion_only_files(pickle_file_path, frame_directory)

                # 앞부분만 no_emotion인 경우 보간 처리
                elif 'no_emotion' in emotion_data.values():
                    print(f"'no_emotion' found in {pickle_file_path}. Applying interpolation...")
                    interpolated_data = interpolate_no_emotion_at_start(emotion_data, total_frames)

                    # 보간된 데이터로 피클 파일을 다시 저장
                    with open(pickle_file_path, 'wb') as f:
                        pickle.dump(interpolated_data, f)
                    print(f"Interpolation completed and saved for {pickle_file_path}")

            except Exception as e:
                print(f"Error processing {pickle_file_path}: {e}")

# 사용 예시
pickle_directory = '//content//drive//MyDrive//sns_multimodal_sentiment_pr//total_emotion_label_pkl'
frame_directory = '/content/drive/MyDrive/sns_multimodal_sentiment_pr/total_video_output_frame_files'

# 디렉토리 내 모든 피클 파일을 처리
process_pickle_files_in_directory(pickle_directory, frame_directory)

Deleting pickle file: //content//drive//MyDrive//sns_multimodal_sentiment_pr//total_emotion_label_pkl/emotion_labels_clip1.pkl
Deleting video frame directory: /content/drive/MyDrive/sns_multimodal_sentiment_pr/total_video_output_frame_files/clip1_frame
Deleting pickle file: //content//drive//MyDrive//sns_multimodal_sentiment_pr//total_emotion_label_pkl/emotion_labels_clip2.pkl
Deleting video frame directory: /content/drive/MyDrive/sns_multimodal_sentiment_pr/total_video_output_frame_files/clip2_frame
Deleting pickle file: //content//drive//MyDrive//sns_multimodal_sentiment_pr//total_emotion_label_pkl/emotion_labels_clip3.pkl
Deleting video frame directory: /content/drive/MyDrive/sns_multimodal_sentiment_pr/total_video_output_frame_files/clip3_frame
'no_emotion' found in //content//drive//MyDrive//sns_multimodal_sentiment_pr//total_emotion_label_pkl/emotion_labels_clip4.pkl. Applying interpolation...
Interpolated no_emotion from frame 1 to 155 using frame 156 (emotion: dislike)
Interpol

In [ ]:
# 피클 파일에서 특정 프레임의 감정 라벨을 수정하는 함수
def modify_emotion_labels_in_pickle(pickle_file_path, frame_start, frame_end, new_emotion):
    try:
        # 피클 파일 읽기
        with open(pickle_file_path, 'rb') as f:
            emotion_data = pickle.load(f)

        # 지정된 범위 내의 프레임들의 감정 라벨을 수정
        for frame_number in range(frame_start, frame_end + 1):
            if emotion_data.get(frame_number) == 'no_emotion':
                emotion_data[frame_number] = new_emotion
                print(f"Frame {frame_number}: 'no_emotion' changed to '{new_emotion}'")

        # 수정된 감정 라벨을 피클 파일에 다시 저장
        with open(pickle_file_path, 'wb') as f:
            pickle.dump(emotion_data, f)
        print(f"Modified pickle file saved: {pickle_file_path}")

    except Exception as e:
        print(f"Error processing {pickle_file_path}: {e}")

# 예시 사용
pickle_file_path = '/content/drive/MyDrive/sns_multimodal_sentiment_pr/total_emotion_label_pkl/emotion_labels_clip136.pkl'

# 287번부터 289번 프레임의 감정 라벨을 'happy'로 변경
modify_emotion_labels_in_pickle(pickle_file_path, frame_start=2171, frame_end=2180, new_emotion='happy')

Frame 2171: 'no_emotion' changed to 'happy'
Frame 2172: 'no_emotion' changed to 'happy'
Frame 2173: 'no_emotion' changed to 'happy'
Frame 2174: 'no_emotion' changed to 'happy'
Frame 2175: 'no_emotion' changed to 'happy'
Frame 2176: 'no_emotion' changed to 'happy'
Frame 2177: 'no_emotion' changed to 'happy'
Frame 2178: 'no_emotion' changed to 'happy'
Frame 2179: 'no_emotion' changed to 'happy'
Frame 2180: 'no_emotion' changed to 'happy'
Modified pickle file saved: /content/drive/MyDrive/sns_multimodal_sentiment_pr/total_emotion_label_pkl/emotion_labels_clip136.pkl


## 통합된 피클파일과 감정라벨 매칭 안되는 파일 삭제 및 개수 비교

In [ ]:
def extract_clip_number(file_or_dir_name, prefix, suffix):
    """파일이나 디렉토리 이름에서 클립 번호를 추출하는 함수."""
    if file_or_dir_name.startswith(prefix) and file_or_dir_name.endswith(suffix):
        return file_or_dir_name[len(prefix):-len(suffix)]
    return None

def clean_and_compare(pickle_dir, frame_dir):
    # 두 디렉토리의 파일 리스트를 가져옴
    pickle_files = [f for f in os.listdir(pickle_dir) if f.startswith("emotion_labels_clip") and f.endswith(".pkl")]
    frame_dirs = [d for d in os.listdir(frame_dir) if d.startswith("clip") and d.endswith("_frame")]

    # 피클 파일과 프레임 디렉토리에서 클립 번호 추출
    pickle_clip_numbers = set(extract_clip_number(f, "emotion_labels_clip", ".pkl") for f in pickle_files)
    frame_clip_numbers = set(extract_clip_number(d, "clip", "_frame") for d in frame_dirs)

    # 피클 파일과 프레임 디렉토리 간의 클립 번호 매칭 여부 확인
    unmatched_pickle_clips = pickle_clip_numbers - frame_clip_numbers
    unmatched_frame_clips = frame_clip_numbers - pickle_clip_numbers

    # 매칭되지 않는 피클 파일과 프레임 디렉토리를 삭제
    for file in pickle_files:
        clip_number = extract_clip_number(file, "emotion_labels_clip", ".pkl")
        if clip_number in unmatched_pickle_clips:
            os.remove(os.path.join(pickle_dir, file))
            print(f"삭제된 피클 파일: {file}")

    for dir in frame_dirs:
        clip_number = extract_clip_number(dir, "clip", "_frame")
        if clip_number in unmatched_frame_clips:
            shutil.rmtree(os.path.join(frame_dir, dir))
            print(f"삭제된 프레임 디렉토리: {dir}")

    # 최종적으로 남은 파일 및 디렉토리 개수를 비교
    remaining_pickle_files = len([f for f in os.listdir(pickle_dir) if f.startswith("emotion_labels_clip") and f.endswith(".pkl")])
    remaining_frame_dirs = len([d for d in os.listdir(frame_dir) if d.startswith("clip") and d.endswith("_frame")])

    # 결과 출력
    if remaining_pickle_files == remaining_frame_dirs:
        print(f"피클 파일과 프레임 디렉토리의 개수가 동일합니다: {remaining_pickle_files}개")
    else:
        print(f"피클 파일과 프레임 디렉토리의 개수가 다릅니다. 피클 파일: {remaining_pickle_files}개, 프레임 디렉토리: {remaining_frame_dirs}개")

# 사용 예시
pickle_directory_path = '//content//drive//MyDrive//sns_multimodal_sentiment_pr//total_emotion_label_pkl'  # 피클 파일이 저장된 디렉토리 경로
frame_directory_path = '/content/drive/MyDrive/sns_multimodal_sentiment_pr/total_video_output_frame_files'   # 프레임 디렉토리가 저장된 디렉토리 경로

clean_and_compare(pickle_directory_path, frame_directory_path)

삭제된 프레임 디렉토리: clip130_frame
피클 파일과 프레임 디렉토리의 개수가 동일합니다: 1312개


## 피클파일에 감정 라벨 종류 체크한 후 다른 단어로 교체
##+) 중립 라벨/손상된 피클 파일/ 미처 삭제되지 못한 no_emotion 삭제
### 무조건 중립 들어간 클립자체를 삭제해야됨 프레임 단위로 삭제하면 라벨이 꼬인다..

In [ ]:
'''
 중립이 들어간 피클과 매칭되는 중립 비디오 클립 자체를 제거
'''
# 감정 라벨 교체 사전
replace_dict = {
    'angry': '0',
    'contempt': '1',
    'dislike': '1',
    'fear': '2',
    'happy': '3',
    'sad': '4',
    'surprise': '5',
    'neutral': '6',
    'no_emotion' : '',  # no_emotion 제거
    '중립': '6',
    '분노': '0',
    '혐오': '1',
    '공포': '2',
    '행복': '3',
    '슬픔': '4',
    '놀람': '5'
}

# 감정 라벨을 교체하는 함수
def replace_emotion_labels_in_pickle(pickle_file_path, replace_dict):
    try:
        with open(pickle_file_path, 'rb') as f:
            data = pickle.load(f)

        # 감정 라벨 교체 및 변환
        new_data = {}
        for key, value in data.items():
            # 인코딩 표준화 및 감정 라벨 변환
            normalized_label = unicodedata.normalize('NFC', value)
            replaced_label = replace_dict.get(normalized_label, normalized_label)  # 변환되지 않는 라벨은 그대로 둠
            new_data[key] = replaced_label

        return new_data

    except (pickle.UnpicklingError, EOFError, AttributeError, ValueError) as e:
        print(f"Error reading pickle file {pickle_file_path}: {e}")
        return None

# 디렉토리 내 모든 피클 파일을 처리하고 출력하는 코드
pickle_directory_path = '//content//drive//MyDrive//sns_multimodal_sentiment_pr//total_emotion_label_pkl'
frame_directory_path = '/content/drive/MyDrive/sns_multimodal_sentiment_pr/total_video_output_frame_files'

# 모든 피클 파일에서 고유한 감정 라벨을 저장할 집합
all_pickle_files = sorted(os.listdir(pickle_directory_path))
all_unique_emotions = set()

# 중립, no_emotion 또는 '6' 감정 라벨이 포함된 피클 파일과 프레임 디렉토리 삭제
for filename in all_pickle_files:
    if filename.endswith('.pkl'):
        pickle_file_path = os.path.join(pickle_directory_path, filename)

        # 감정 라벨을 교체한 데이터를 가져옴
        updated_data = replace_emotion_labels_in_pickle(pickle_file_path, replace_dict)

        if updated_data:
            # 고유한 감정 라벨을 집합에 추가
            all_unique_emotions.update(updated_data.values())

            # 피클 파일에 변경된 데이터를 다시 저장
            with open(pickle_file_path, 'wb') as f:
                pickle.dump(updated_data, f)

# 최종적으로 남은 피클 파일에서 감정 라벨을 출력
print("\n전체 고유한 감정 라벨들:")
print(all_unique_emotions)


전체 고유한 감정 라벨들:
{'1', '0', '4', '2', '3', '5'}


## 감정별 라벨 개수

In [ ]:
from collections import Counter

# 피클 파일 읽어오는 함수
def load_pickle_file(pickle_file_path):
    try:
        with open(pickle_file_path, 'rb') as f:
            data = pickle.load(f)
            return data
    except Exception as e:
        print(f"Error loading pickle file {pickle_file_path}: {e}")
        return None

# 디렉토리 내 모든 피클 파일의 감정 라벨을 카운트하는 함수
def count_emotion_labels_in_directory(pickle_directory):
    all_pickle_files = sorted(os.listdir(pickle_directory))  # 디렉토리 내 파일 리스트 정렬
    total_emotion_counts = Counter()  # 감정 라벨 개수를 카운트할 Counter 객체

    # 디렉토리 내 모든 피클 파일을 순회하며 감정 라벨 개수 카운트
    for filename in all_pickle_files:
        if filename.endswith('.pkl'):  # 피클 파일만 처리
            pickle_file_path = os.path.join(pickle_directory, filename)
            emotion_data = load_pickle_file(pickle_file_path)

            if emotion_data is not None:
                # 피클 파일 내의 감정 라벨 개수를 Counter에 추가
                total_emotion_counts.update(emotion_data.values())
                print(f"Processed {filename}")

    # 각 감정 라벨의 개수 출력
    print("\nTotal counts for each emotion label:")
    for emotion, count in total_emotion_counts.items():
        print(f"Label: {emotion}, Count: {count}")

# 사용 예시
pickle_directory = '//content//drive//MyDrive//sns_multimodal_sentiment_pr//total_emotion_label_pkl'

# 디렉토리 내 모든 피클 파일의 감정 라벨 카운트
count_emotion_labels_in_directory(pickle_directory)

Processed emotion_labels_clip1.pkl
Processed emotion_labels_clip10.pkl
Processed emotion_labels_clip100.pkl
Processed emotion_labels_clip1000.pkl
Processed emotion_labels_clip1001.pkl
Processed emotion_labels_clip1002.pkl
Processed emotion_labels_clip1003.pkl
Processed emotion_labels_clip1004.pkl
Processed emotion_labels_clip1005.pkl
Processed emotion_labels_clip1006.pkl
Processed emotion_labels_clip1007.pkl
Processed emotion_labels_clip1008.pkl
Processed emotion_labels_clip1009.pkl
Processed emotion_labels_clip101.pkl
Processed emotion_labels_clip1010.pkl
Processed emotion_labels_clip1011.pkl
Processed emotion_labels_clip1012.pkl
Processed emotion_labels_clip1013.pkl
Processed emotion_labels_clip1014.pkl
Processed emotion_labels_clip1015.pkl
Processed emotion_labels_clip1016.pkl
Processed emotion_labels_clip1017.pkl
Processed emotion_labels_clip1018.pkl
Processed emotion_labels_clip1019.pkl
Processed emotion_labels_clip102.pkl
Processed emotion_labels_clip1020.pkl
Processed emotion_la

## 프레임과 라벨 총 개수

In [ ]:
# 피클 파일 읽어오는 함수
def load_pickle_file(pickle_file_path):
    try:
        with open(pickle_file_path, 'rb') as f:
            data = pickle.load(f)
            return data
    except Exception as e:
        print(f"Error loading pickle file {pickle_file_path}: {e}")
        return None

# 디렉토리 내 총 비디오 프레임 개수와 총 감정 라벨 개수를 카운트하는 함수
def count_total_frames_and_labels_in_directory(pickle_directory):
    all_pickle_files = sorted(os.listdir(pickle_directory))  # 디렉토리 내 파일 리스트 정렬
    total_frame_count = 0  # 총 비디오 프레임 개수
    total_label_count = 0  # 총 감정 라벨 개수

    # 디렉토리 내 모든 피클 파일을 순회하며 비디오 프레임과 감정 라벨 개수 카운트
    for filename in all_pickle_files:
        if filename.endswith('.pkl'):  # 피클 파일만 처리
            pickle_file_path = os.path.join(pickle_directory, filename)
            emotion_data = load_pickle_file(pickle_file_path)

            if emotion_data is not None:
                # 총 프레임 개수는 데이터의 키로부터 계산 (키가 프레임 번호인 경우)
                frame_count = len(emotion_data)
                total_frame_count += frame_count

                # 총 감정 라벨 개수는 데이터의 값으로부터 계산
                total_label_count += len(emotion_data.values())
                print(f"{filename} contains {frame_count} frames and {len(emotion_data.values())} labels.")

    # 총 비디오 프레임 개수와 총 감정 라벨 개수 출력
    print(f"\nTotal video frames across all files: {total_frame_count}")
    print(f"Total emotion labels across all files: {total_label_count}")

# 사용 예시
pickle_directory = '//content//drive//MyDrive//sns_multimodal_sentiment_pr//total_emotion_label_pkl'

# 디렉토리 내 총 비디오 프레임 개수와 총 감정 라벨 개수 카운트
count_total_frames_and_labels_in_directory(pickle_directory)

emotion_labels_clip1.pkl contains 165 frames and 165 labels.
emotion_labels_clip10.pkl contains 135 frames and 135 labels.
emotion_labels_clip100.pkl contains 105 frames and 105 labels.
emotion_labels_clip1000.pkl contains 150 frames and 150 labels.
emotion_labels_clip1001.pkl contains 1509 frames and 1509 labels.
emotion_labels_clip1002.pkl contains 120 frames and 120 labels.
emotion_labels_clip1003.pkl contains 150 frames and 150 labels.
emotion_labels_clip1004.pkl contains 120 frames and 120 labels.
emotion_labels_clip1005.pkl contains 150 frames and 150 labels.
emotion_labels_clip1006.pkl contains 120 frames and 120 labels.
emotion_labels_clip1007.pkl contains 135 frames and 135 labels.
emotion_labels_clip1008.pkl contains 135 frames and 135 labels.
emotion_labels_clip1009.pkl contains 135 frames and 135 labels.
emotion_labels_clip101.pkl contains 90 frames and 90 labels.
emotion_labels_clip1010.pkl contains 120 frames and 120 labels.
emotion_labels_clip1011.pkl contains 135 frames

## 테스트 체크

In [ ]:
# 피클 파일을 읽어서 데이터를 반환하는 함수
def load_pickle_file(pickle_file_path):
    with open(pickle_file_path, 'rb') as f:  # 'rb' 모드로 피클 파일을 읽기
        data = pickle.load(f)  # 피클 파일의 내용을 로드
    return data

# 사용 예시
pickle_file_path = '//content//drive//MyDrive//sns_multimodal_sentiment_pr//total_emotion_label_pkl//emotion_labels_clip136.pkl'  # 피클 파일 경로
data = load_pickle_file(pickle_file_path)

# 데이터 출력
print(data)

{0: '5', 1: '5', 2: '5', 3: '5', 4: '5', 5: '5', 6: '5', 7: '5', 8: '5', 9: '5', 10: '5', 11: '5', 12: '5', 13: '5', 14: '5', 15: '5', 16: '5', 17: '5', 18: '5', 19: '5', 20: '5', 21: '5', 22: '5', 23: '5', 24: '5', 25: '5', 26: '5', 27: '5', 28: '5', 29: '5', 30: '5', 31: '5', 32: '5', 33: '5', 34: '5', 35: '5', 36: '5', 37: '5', 38: '5', 39: '5', 40: '5', 41: '5', 42: '5', 43: '5', 44: '5', 45: '5', 46: '5', 47: '5', 48: '5', 49: '5', 50: '5', 51: '5', 52: '5', 53: '5', 54: '5', 55: '5', 56: '5', 57: '5', 58: '5', 59: '5', 60: '5', 61: '5', 62: '5', 63: '5', 64: '5', 65: '5', 66: '5', 67: '5', 68: '5', 69: '5', 70: '5', 71: '5', 72: '5', 73: '5', 74: '5', 75: '5', 76: '5', 77: '5', 78: '5', 79: '5', 80: '5', 81: '5', 82: '5', 83: '5', 84: '5', 85: '5', 86: '5', 87: '5', 88: '5', 89: '5', 90: '5', 91: '5', 92: '5', 93: '5', 94: '5', 95: '5', 96: '5', 97: '5', 98: '5', 99: '5', 100: '5', 101: '5', 102: '5', 103: '5', 104: '5'}


# 통합 비디오 데이터 트랜스포머 입력 형태로 변환
## 비디오 프레임 추출 함수인데 이 코드는 표정 데이터 전처리 단계로 옮겨야 될듯
## 비디오 프레임 속도 잊지말고 고려하기  

In [ ]:
# GPU 사용 여부 확인
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cuda


In [ ]:
import torch.nn as nn
from PIL import Image
from torchvision import models, transforms



# ResNet 모델에서 마지막 fully connected layer를 제거하고 특징 벡터 추출
class ResNetFeatureExtractor(nn.Module):
    def __init__(self):
        super(ResNetFeatureExtractor, self).__init__()
        # 사전 학습된 ResNet50 모델 로드
        self.resnet = models.resnet50(pretrained=True)
        # ResNet의 마지막 레이어(fc)를 제거
        self.feature_extractor = nn.Sequential(*list(self.resnet.children())[:-1])

    def forward(self, x):
        # ResNet의 마지막 fc 레이어 이전까지 통과한 특징 벡터 (2048차원)
        features = self.feature_extractor(x)
        # (1, 2048, 1, 1) 형태에서 (1, 2048)로 차원 축소
        features = features.view(features.size(0), -1)
        return features

# 선형 변환 레이어 (2048차원 -> 768차원으로 줄이기 위한 Fully Connected Layer)
class FeatureReducer(nn.Module):
    def __init__(self):
        super(FeatureReducer, self).__init__()
        self.fc = nn.Linear(2048, 768)  # 2048차원 특징 벡터를 768차원으로 축소하는 레이어

    def forward(self, x):
        return self.fc(x)

# 이미지 전처리 (224x224 크기로 변환 및 정규화)
image_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# ResNet 모델 로드 및 GPU로 이동
feature_extractor = ResNetFeatureExtractor().to(device)
reducer = FeatureReducer().to(device)

# 특정 프레임 디렉토리에서 프레임을 처리하고 임베딩 벡터와 라벨을 저장하는 함수
def process_single_frame_dir(frames_dir, pickle_file_path, embeddings_dict):
    # 프레임 파일 리스트 정렬
    frame_files = sorted(os.listdir(frames_dir), key=lambda x: int(x.split('_')[-1].split('.')[0]))  # 숫자 기준 정렬

    # 감정 라벨 피클 파일 로드
    with open(pickle_file_path, 'rb') as f:
        emotion_labels = pickle.load(f)

    # 각 프레임마다 ResNet을 통해 특징 벡터 추출 및 768차원으로 변환
    for frame_file in frame_files:
        frame_number = int(frame_file.split('_')[-1].split('.')[0])  # 예: frame_1.jpg에서 번호 추출
        frame_full_path = os.path.join(frames_dir, frame_file)

        # 이미지 로드 및 전처리
        image = Image.open(frame_full_path).convert('RGB')
        image_tensor = image_transform(image).unsqueeze(0).to(device)  # (1, 3, 224, 224), GPU로 이동

        # ResNet을 사용해 특징 벡터 추출 (GPU에서 연산)
        with torch.no_grad():
            frame_features = feature_extractor(image_tensor)  # (1, 2048)

        # 2048차원 특징 벡터를 768차원으로 축소 (GPU에서 연산)
        with torch.no_grad():
            embedding_vector = reducer(frame_features).cpu().numpy().flatten()  # 768차원 벡터, CPU로 다시 이동

        # 감정 라벨과 매칭
        emotion_label = emotion_labels.get(frame_number, "no_label")  # 해당 프레임 번호의 감정 라벨

        # 프레임 번호와 768차원 임베딩 벡터, 감정 라벨을 딕셔너리에 저장
        embeddings_dict[frame_number] = {
            'embedding': embedding_vector,
            'label': emotion_label
        }

# 모든 하위 디렉토리에서 프레임을 처리하고 임베딩 벡터와 라벨을 매칭하여 피클 파일로 저장하는 함수
def process_all_frame_dirs(frames_base_dir, pickle_base_dir, output_dir):
    # output_dir이 존재하지 않으면 생성
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
        print(f"Created output directory: {output_dir}")

    # 기본 디렉토리 내의 모든 하위 디렉토리를 처리
    for subdir in tqdm(os.listdir(frames_base_dir)):
        frames_dir = os.path.join(frames_base_dir, subdir)

        if os.path.isdir(frames_dir):
            # clip105_frame에서 clip105 추출
            clip_number = subdir.split('_')[0].replace('clip', '')

            # 감정 라벨 피클 파일 경로 설정 (emotion_labels_clipX.pkl 형식)
            pickle_file_path = os.path.join(pickle_base_dir, f"emotion_labels_clip{clip_number}.pkl")

            # 피클 파일이 존재하는지 확인하고 없으면 스킵
            if not os.path.exists(pickle_file_path):
                print(f"Skipping: emotion_labels_clip{clip_number}.pkl not found.")
                continue

            # 최종 저장할 피클 파일 경로 설정
            output_pkl_path = os.path.join(output_dir, f"{subdir}_embeddings_with_labels.pkl")

            # 이미 처리된 파일이 있는지 확인하고 스킵
            if os.path.exists(output_pkl_path):
                print(f"Skipping: {output_pkl_path} already exists.")
                continue

            # 임베딩 벡터와 라벨을 저장할 딕셔너리 초기화
            embeddings_dict = {}

            # 각 하위 디렉토리의 프레임을 처리
            process_single_frame_dir(frames_dir, pickle_file_path, embeddings_dict)

            # 임베딩 벡터와 라벨 딕셔너리를 피클 파일로 저장
            with open(output_pkl_path, 'wb') as f:
                pickle.dump(embeddings_dict, f)

            print(f"Processed {frames_dir}, embeddings saved to {output_pkl_path}")
        else:
            print(f"Skipping: {frames_dir} is not a directory.")



/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 176MB/s]


In [ ]:
# 사용 예시
frames_base_dir = '//content//drive//MyDrive//sns_multimodal_sentiment_pr//total_video_output_frame_files'
pickle_base_dir = '/content/drive/MyDrive/sns_multimodal_sentiment_pr/total_emotion_label_pkl'
output_dir = '/content/drive/MyDrive/sns_multimodal_sentiment_pr/final_embeddings_with_labels'

# 모든 하위 디렉토리의 프레임을 처리하고 피클 파일로 저장 (GPU 사용)
process_all_frame_dirs(frames_base_dir, pickle_base_dir, output_dir)

 22%|██▏       | 167/772 [00:04<00:10, 57.32it/s]

Skipping: /content/drive/MyDrive/sns_multimodal_sentiment_pr/final_embeddings_with_labels/clip4_frame_embeddings_with_labels.pkl already exists.
Skipping: /content/drive/MyDrive/sns_multimodal_sentiment_pr/final_embeddings_with_labels/clip5_frame_embeddings_with_labels.pkl already exists.
Skipping: /content/drive/MyDrive/sns_multimodal_sentiment_pr/final_embeddings_with_labels/clip6_frame_embeddings_with_labels.pkl already exists.
Skipping: /content/drive/MyDrive/sns_multimodal_sentiment_pr/final_embeddings_with_labels/clip15_frame_embeddings_with_labels.pkl already exists.
Skipping: /content/drive/MyDrive/sns_multimodal_sentiment_pr/final_embeddings_with_labels/clip16_frame_embeddings_with_labels.pkl already exists.
Skipping: /content/drive/MyDrive/sns_multimodal_sentiment_pr/final_embeddings_with_labels/clip17_frame_embeddings_with_labels.pkl already exists.
Skipping: /content/drive/MyDrive/sns_multimodal_sentiment_pr/final_embeddings_with_labels/clip18_frame_embeddings_with_labels.p

 26%|██▋       | 203/772 [02:03<07:30,  1.26it/s]

Processed //content//drive//MyDrive//sns_multimodal_sentiment_pr//total_video_output_frame_files/clip269_frame, embeddings saved to /content/drive/MyDrive/sns_multimodal_sentiment_pr/final_embeddings_with_labels/clip269_frame_embeddings_with_labels.pkl


 26%|██▋       | 204/772 [03:43<16:12,  1.71s/it]

...

Processed //content//drive//MyDrive//sns_multimodal_sentiment_pr//total_video_output_frame_files/clip915_frame, embeddings saved to /content/drive/MyDrive/sns_multimodal_sentiment_pr/final_embeddings_with_labels/clip915_frame_embeddings_with_labels.pkl


100%|█████████▉| 770/772 [2:05:24<00:01,  1.41it/s]

Processed //content//drive//MyDrive//sns_multimodal_sentiment_pr//total_video_output_frame_files/clip916_frame, embeddings saved to /content/drive/MyDrive/sns_multimodal_sentiment_pr/final_embeddings_with_labels/clip916_frame_embeddings_with_labels.pkl


100%|██████████| 772/772 [2:05:25<00:00,  9.75s/it]

Processed //content//drive//MyDrive//sns_multimodal_sentiment_pr//total_video_output_frame_files/clip917_frame, embeddings saved to /content/drive/MyDrive/sns_multimodal_sentiment_pr/final_embeddings_with_labels/clip917_frame_embeddings_with_labels.pkl
Skipping: emotion_labels_clip918.pkl not found.


## 임베딩 파일 개수 체크

In [ ]:
import os

def count_files_in_directory(directory_path):
    try:
        # 디렉토리 내 모든 파일의 개수를 카운트
        file_count = len([name for name in os.listdir(directory_path) if os.path.isfile(os.path.join(directory_path, name))])
        return file_count
    except Exception as e:
        print(f"Error: {e}")
        return 0

# 사용 예시
directory_path ='/content/drive/MyDrive/sns_multimodal_sentiment_pr/final_embeddings_with_labels' # 디렉토리 경로를 입력하세요
print(f"Total number of files: {count_files_in_directory(directory_path)}")

Total number of files: 553


## 임베딩 파일 테스트 체크

In [ ]:
import pickle

def load_pickle_file(file_path):
    try:
        with open(file_path, 'rb') as f:
            data = pickle.load(f)
        print("Pickle file loaded successfully.")
        return data
    except Exception as e:
        print(f"Error loading pickle file: {e}")
        return None

# 사용 예시
file_path = '/content/drive/MyDrive/sns_multimodal_sentiment_pr/final_embeddings_with_labels/clip871_frame_embeddings_with_labels.pkl'  # 피클 파일 경로 입력
data = load_pickle_file(file_path)

# 로드한 데이터 출력
if data:
    print(data)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
        1.10349737e-01,  1.07189856e-01,  3.17391694e-01, -1.66605964e-01,
        1.57196209e-01,  3.69014777e-03,  1.45861626e-01, -4.94864613e-01,
        4.76867035e-02, -3.15647453e-01, -8.48601162e-02,  3.36679146e-02,
       -3.62602293e-01,  1.55366153e-01, -5.58419287e-01, -4.27458942e-01,
       -1.55080289e-01, -5.82685173e-02, -5.10658696e-02, -2.68213779e-01,
       -1.19633861e-01, -7.09021270e-01, -2.12402508e-01, -3.63928020e-01,
       -5.41326284e-01, -2.69329011e-01,  3.10261250e-01, -1.04470618e-01,
        4.54430103e-01,  8.01227689e-02,  1.81936808e-02,  2.62923151e-01,
        1.17017061e-01,  9.24026296e-02,  3.20205629e-01, -2.58670062e-01,
       -1.97039247e-01,  2.59880334e-01,  2.09110770e-02,  1.11729778e-01,
       -2.60081887e-01,  3.51095736e-01,  9.14596096e-02,  2.32220441e-01,
       -4.34797972e-01,  4.37289298e-01, -5.25754169e-02,  3.03396910e-01,
       -9.70090032e-02, -4.20048907e-02,  3.69029492e-01, -1.349

# 피클파일에서 오류가 생겨서 감정라벨이 숫자로 변환이 안돼 다시 임베딩 파일에서 변환 진행

## 감정라벨 교체

In [ ]:
# 감정 라벨 교체 사전 (텍스트 -> 숫자)
replace_dict = {
    'angry': '0',
    'contempt': '1',
    'dislike': '1',
    'fear': '2',
    'happy': '3',
    'sad': '4',
    'surprise': '5',
    '분노': '0',
    '혐오': '1',
    '공포': '2',
    '행복': '3',
    '슬픔': '4',
    '놀람': '5'
}

# 피클 파일의 감정 라벨을 숫자로 변환하는 함수
def replace_emotion_labels_in_pickle(pickle_file_path, replace_dict):
    try:
        # 피클 파일 열기
        with open(pickle_file_path, 'rb') as f:
            data = pickle.load(f)

        # 감정 라벨 변환 (딕셔너리 안의 값들 교체)
        for frame_number, frame_data in data.items():
            if 'label' in frame_data:  # 'label'이라는 키가 있는 경우에만
                emotion_label = frame_data['label']
                # 감정 라벨을 숫자로 교체
                new_label = replace_dict.get(emotion_label, emotion_label)  # 없으면 원래 값 유지
                data[frame_number]['label'] = new_label

        # 변환된 데이터를 다시 피클 파일로 저장
        with open(pickle_file_path, 'wb') as f:
            pickle.dump(data, f)

        print(f"Updated emotion labels in {pickle_file_path}")

    except Exception as e:
        print(f"Error processing {pickle_file_path}: {e}")

# 디렉토리 내 모든 피클 파일을 처리하는 함수
def process_pickle_files_in_directory(pickle_directory, replace_dict):
    for filename in os.listdir(pickle_directory):
        if filename.endswith('.pkl'):  # 피클 파일만 처리
            pickle_file_path = os.path.join(pickle_directory, filename)
            replace_emotion_labels_in_pickle(pickle_file_path, replace_dict)

# 사용 예시
pickle_directory = '/content/drive/MyDrive/sns_multimodal_sentiment_pr/final_embeddings_with_labels'  # 피클 파일이 있는 디렉토리 경로

# 디렉토리 내 모든 피클 파일 처리
process_pickle_files_in_directory(pickle_directory, replace_dict)

In [ ]:
# 디렉토리 내 모든 피클 파일에서 고유한 감정 라벨을 추출하는 함수
def extract_unique_labels_from_pickle_files(pickle_directory):
    unique_labels = set()  # 고유한 감정 라벨을 저장할 집합

    # 디렉토리 내 모든 피클 파일을 처리
    for filename in os.listdir(pickle_directory):
        if filename.endswith('.pkl'):  # 피클 파일만 처리
            pickle_file_path = os.path.join(pickle_directory, filename)
            try:
                # 피클 파일 열기
                with open(pickle_file_path, 'rb') as f:
                    data = pickle.load(f)

                # 감정 라벨 추출
                for frame_data in data.values():
                    if 'label' in frame_data:
                        unique_labels.add(frame_data['label'])  # 고유한 라벨을 집합에 추가

            except Exception as e:
                print(f"Error processing {pickle_file_path}: {e}")

    return unique_labels

# 사용 예시
pickle_directory = '/content/drive/MyDrive/sns_multimodal_sentiment_pr/final_embeddings_with_labels'  # 피클 파일이 있는 디렉토리 경로

# 고유한 감정 라벨 추출
unique_labels = extract_unique_labels_from_pickle_files(pickle_directory)

# 고유한 감정 라벨 출력
print("Unique emotion labels found:")
print(unique_labels)

Unique emotion labels found:
{'2', '0', '4', '5', '1', '3'}
